In [53]:
!pip install bertopic scikit-learn nltk umap-learn sentence-transformers langdetect pdfplumber hdbscan

In [61]:
!pip install --upgrade bertopic

In [63]:
import numpy as np
import nltk
import re
from umap import UMAP
from bertopic import BERTopic
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE, trustworthiness
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from scipy.stats import spearmanr
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
#from bertopic.vectorizers import ClassTFIDF
from bertopic.representation import KeyBERTInspired
from tqdm import tqdm
import re, unicodedata, json
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from langdetect import detect
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import numpy as np
from tqdm import tqdm
import pdfplumber
from nltk import pos_tag
from sklearn.preprocessing import normalize
from hdbscan import HDBSCAN

import warnings
warnings.filterwarnings("ignore", category=SyntaxWarning)

In [8]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
PDF_PATH = "/content/04MC01042025636879CB4F0548F490DC312D96B7D2C1.pdf"

In [11]:
# -------------------------------
# 1. Extract raw text using pdfplumber
# -------------------------------

pages = []
with pdfplumber.open(PDF_PATH) as pdf:

    # Define 0-index page ranges
    selected_page_indices = (
        list(range(2, 30)) +      # pages 3–30
        list(range(48, 50)) +     # pages 49–50
        list(range(65, 68)) +     # pages 66–68
        list(range(69, 71))       # pages 70–71
    )

    for idx in selected_page_indices:
        if idx < len(pdf.pages):
            text = pdf.pages[idx].extract_text() or ""
            pages.append(text)

raw = "\n".join(pages)

In [13]:
# -------------------------------
# 2. Normalize Unicode + whitespace
# -------------------------------
def normalize(text):
    text = unicodedata.normalize("NFKC", text)
    text = re.sub(r"\r\n?", "\n", text)
    text = re.sub(r"[ \t]+", " ", text)
    return text.strip()

raw = normalize(raw)

In [15]:
# -------------------------------
# 3. Remove boilerplate / headers / footers
#    These patterns come directly from your PDF
# -------------------------------
patterns_to_remove = [
    r"Caution: RBI never sends.*",        # disclaimer footer
    r"Financial Inclusion.*Mumbai.*",     # header on page 1
    r"हिंदी आसान है.*",                  # Hindi slogan
    r"RBI/\d{4}-\d{2}/\d+.*",             # circular reference lines
    r"^\s*\d+\s*$",                       # standalone page numbers
]

for pat in patterns_to_remove:
    raw = re.sub(pat, "", raw, flags=re.I | re.M | re.S)

In [22]:
# -------------------------------
# 4. Fix line breaks (VERY important for PDFs)
#    If a line ends with no punctuation, join it with next line.
# -------------------------------
processed_lines = []
lines = raw.split("\n")

buffer = ""
for line in lines:
    stripped = line.strip()
    if not stripped:
        continue

    if buffer == "":
        buffer = stripped
        continue

    # If previous line doesn't end with punctuation and current starts lowercase/digit
    if (not re.search(r"[.!?]$", buffer)) and re.match(r"^[a-z0-9]", stripped):
        buffer += " " + stripped
    else:
        processed_lines.append(buffer)
        buffer = stripped

if buffer:
    processed_lines.append(buffer)

clean_text = "\n".join(processed_lines)

In [24]:
# -------------------------------
# 5. Sentence tokenization (NLTK)
# -------------------------------
sentences = sent_tokenize(clean_text)

In [32]:
def remove_bullets(s):

    # Remove bullets at the START of sentence
    s = re.sub(r"^\s*[\•\-\*]\s*", "", s)                          # • bullet
    s = re.sub(r"^\s*[a-zA-Z]\s*[\.\)]\s*", "", s)                # a. , a)
    s = re.sub(r"^\s*\(?[a-zA-Z]\)\s*", "", s)                    # (a)
    s = re.sub(r"^\s*\(?[ivxIVX]+\)\s*", "", s)                   # (i), (iv)
    s = re.sub(r"^\s*[ivxIVX]+\s*[\.\)]\s*", "", s)               # i. , iv)

    # Remove bullets appearing IN THE MIDDLE of sentences
    s = re.sub(r"\s+[a-zA-Z]\s*[\.\)]\s+", " ", s)                # a. b. c.
    s = re.sub(r"\s+\(?[a-zA-Z]\)\s+", " ", s)                    # (a) (b)
    s = re.sub(r"\s+[ivxIVX]+\s*[\.\)]\s+", " ", s)               # i. ii. iii.
    s = re.sub(r"\s+\(?[ivxIVX]+\)\s+", " ", s)                   # (i) (ii)

    return s.strip()


In [33]:
import re
from nltk import word_tokenize, pos_tag

# -------------------------------
# CONFIGURATION FOR SENTENCE FILTERING
# -------------------------------
MIN_WORDS = 3         # minimum number of words
MIN_CHARS = 15        # minimum characters
USE_POS_VERB_CHECK = False    # make True if you want only sentences with verbs

WHITELIST_PATTERNS = [
    r"\bCD Ratio\b",
    r"\bSLBC\b",
    r"\bDCC\b",
    r"\bLDM\b",
    r"\bDBT\b",
    r"\b\d{4}\b"  # years like 2019, 2020, etc.
]

def is_whitelisted(s):
    for pat in WHITELIST_PATTERNS:
        if re.search(pat, s, flags=re.I):
            return True
    return False

def has_verb(s):
    """Check if sentence contains at least one verb."""
    try:
        tokens = word_tokenize(s)
        tags = pos_tag(tokens)
        return any(tag.startswith("VB") for _, tag in tags)
    except:
        return False

def keep_sentence(s, min_words=MIN_WORDS, min_chars=MIN_CHARS,
                  verb_check=USE_POS_VERB_CHECK):
    """Apply all rules: minimum words, characters, optional POS-verb check, whitelist."""

    s = s.strip()
    if not s:
        return False

    # Count words
    words = re.findall(r"\S+", s)

    # Word count filter
    if len(words) < min_words and not is_whitelisted(s):
        return False

    # Character length filter
    if len(s) < min_chars and not is_whitelisted(s):
        return False

    # Optional verb check
    if verb_check:
        if not has_verb(s) and not is_whitelisted(s):
            return False

    return True


# -------------------------------
# FINAL CLEANING + FILTERING LOOP
# -------------------------------
processed_sents = []

for s in sentences:
    s = s.strip()

    # Remove section numbers like 2.2.4, 1.1.3, etc.
    s = re.sub(r"^\d+(\.\d+){0,3}\s*", "", s)

    # 🔥 Remove all bullet types
    s = remove_bullets(s)

    # Collapse spaces
    s = re.sub(r"\s+", " ", s)

    # Apply advanced filter
    if keep_sentence(s):
        processed_sents.append(s)


In [35]:

english_sents = []

for s in processed_sents:

    # Try to detect language
    try:
        lang = detect(s)
    except:
        lang = "unknown"

    # Keep English OR whitelisted domain tokens
    if lang.startswith("en") or is_whitelisted(s):
        english_sents.append(s)

In [41]:
english_sents = list(set(english_sents))

In [42]:
# 4) minimal cleaning: remove empty lines
text_sen = [s.strip() for s in english_sents if len(s.strip()) > 0]

In [44]:
# 5) Basic embedding
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(text_sen, convert_to_numpy=True)
print("Embeddings shape:", embeddings.shape)

Embeddings shape: (382, 384)


In [45]:
pca = PCA(n_components=0.85, whiten=True, random_state=42)
embeddings_pca = pca.fit_transform(embeddings)
print("Embeddings shape after PCA:", embeddings_pca.shape)

Embeddings shape after PCA: (382, 73)


In [55]:
embeddings_pca_norm = normalize(embeddings_pca, norm="l2", axis=1)
print("Embeddings shape after normalization:", embeddings_pca_norm.shape)

Embeddings shape after normalization: (382, 73)


In [56]:
# 2) Explain variance summary
explained = pca.explained_variance_ratio_.sum() if 0.85 <= pca.n_components_ else pca.explained_variance_ratio_.sum()
print(f"Total explained variance by {0.85} components: {explained:.3f}")
# show first few ratios
print("Top explained variance ratios (first 10):", pca.explained_variance_ratio_[:10])

Total explained variance by 0.85 components: 0.852
Top explained variance ratios (first 10): [0.08208989 0.07154558 0.04645811 0.04093289 0.03396012 0.02883822
 0.02655612 0.02410786 0.0223011  0.02074771]


In [79]:
from bertopic import BERTopic
from bertopic.dimensionality import BaseDimensionalityReduction

empty_reduction_model = BaseDimensionalityReduction()

In [83]:
# Build a custom HDBSCAN model for BERTopic
# -----------------------
hdbscan_model = HDBSCAN(min_cluster_size=5,    # tune this: 5-20 typical
              min_samples=1,        # lower => more clusters; raise for tighter clusters
              metric='euclidean',   # using Euclidean on normalized PCA approximates cosine
              cluster_selection_method='eom',
              prediction_data=True)

In [84]:
vectorizer_model = CountVectorizer(
    stop_words="english",
    ngram_range=(1, 2),
    min_df=2,                 # ignore extremely rare tokens
    max_df=0.95,              # ignore tokens appearing in >95% docs
)

In [85]:
ctfidf_model = ClassTfidfTransformer(
    reduce_frequent_words=True,   # prevents very common tokens dominating
)


In [86]:
topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    hdbscan_model=hdbscan_model,
    embedding_model=None,
    umap_model=empty_reduction_model,
    calculate_probabilities=True,
    verbose=True
)

In [87]:
topics, probs = topic_model.fit_transform(
    text_sen,
    embeddings=embeddings_pca_norm
)

2025-11-24 16:30:01,994 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-24 16:30:01,998 - BERTopic - Dimensionality - Completed ✓
2025-11-24 16:30:02,001 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-24 16:30:02,116 - BERTopic - Cluster - Completed ✓
2025-11-24 16:30:02,126 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-24 16:30:02,222 - BERTopic - Representation - Completed ✓


In [88]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,160,-1_issues_sponsored_wise_district wise,"[issues, sponsored, wise, district wise, repre...",[programmes Banking Banking Network- Mode of B...
1,0,33,0_portal_upload_block wise_data,"[portal, upload, block wise, data, access, con...",[For Data pertaining to State Government/ Othe...
2,1,15,1_rrbs_cooperative banks_foreign banks_foreign,"[rrbs, cooperative banks, foreign banks, forei...","[RBI, NABARD, all commercial banks including S..."
3,2,15,2_villages_centres_roadmap_banking services,"[villages, centres, roadmap, banking services,...","[Accordingly, SLBC Convenor Banks were advised..."
4,3,15,3_cd ratio_cd_ratio_percent,"[cd ratio, cd, ratio, percent, districts, ssc,...",[SSCs of the DCC should be set up in the distr...
5,4,14,4_dlrc forum_dlrc_forum_forum shall,"[dlrc forum, dlrc, forum, forum shall, solutio...",[As the DLRC forum has public representatives ...
6,5,12,5_seeding_dbt_aadhaar_redressal,"[seeding, dbt, aadhaar, redressal, end, end qu...",[Banks were further advised to ensure that ope...
7,6,10,6_blbc_bankers committee_level bankers_bankers,"[blbc, bankers committee, level bankers, banke...","[Therefore, NABARD has been advised that DDMs ..."
8,7,10,7_calendar_yearly_dates_beginning,"[calendar, yearly, dates, beginning, intimatio...",[SLBC - Yearly Calendar of Meetings To improve...
9,8,8,8_lead district_ldm_ldo_manager ldm,"[lead district, ldm, ldo, manager ldm, support...",[The office of the Lead District Manager (LDM)...


In [89]:
new_topics = topic_model.reduce_outliers(text_sen, topics, probabilities=probs, strategy="distributions")

100%|██████████| 1/1 [00:00<00:00, 11.60it/s]


In [90]:
topic_model.update_topics(text_sen, topics=new_topics)

2025-11-24 16:30:15,174 - BERTopic - WARNING: Using a custom list of topic assignments may lead to errors if topic reduction techniques are used afterwards. Make sure that manually assigning topics is the last step in the pipeline.Note that topic embeddings will also be created through weightedc-TF-IDF embeddings instead of centroid embeddings.


In [92]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,42,0_data_slbc_state_portal,"[data, slbc, state, portal, to, the, controlli...",[For Data pertaining to State Government/ Othe...
1,1,22,1_banks_rrbs_sfbs_foreign,"[banks, rrbs, sfbs, foreign, wos, of, and, rep...","[RBI, NABARD, all commercial banks including S..."
2,2,22,2_banking_villages_services_unbanked,"[banking, villages, services, unbanked, outlet...","[Accordingly, SLBC Convenor Banks were advised..."
3,3,29,3_ratio_cd_districts_percent,"[ratio, cd, districts, percent, the, ssc, be, ...",[SSCs of the DCC should be set up in the distr...
4,4,24,4_dlrc_forum_district_the,"[dlrc, forum, district, the, review, credit, s...",[As the DLRC forum has public representatives ...
5,5,13,5_accounts_dbt_seeding_days,"[accounts, dbt, seeding, days, aadhaar, meetin...",[Banks were further advised to ensure that ope...
6,6,20,6_level_committee_bankers_blbc,"[level, committee, bankers, blbc, block, the, ...","[Therefore, NABARD has been advised that DDMs ..."
7,7,16,7_calendar_dates_meetings_yearly,"[calendar, dates, meetings, yearly, intimation...",[SLBC - Yearly Calendar of Meetings To improve...
8,8,15,8_district_the_ldm_of,"[district, the, ldm, of, collectors, lead, for...",[The office of the Lead District Manager (LDM)...
9,9,12,9_farmers_income_enhancing_as,"[farmers, income, enhancing, as, in, the, lbs,...",[Implementation of Lead Bank Scheme 3.1 Prepar...
